#Loading all packages and datasets

In [0]:
from pyspark.sql.functions import col,isnan,when,count
from pyspark.sql import functions as f
from pyspark.sql import SQLContext
from pyspark.sql.functions import monotonically_increasing_id
from pyspark.sql.functions import isnan, when, count, col, isnull, percent_rank
from pyspark.sql.functions import min
from pyspark.sql.functions import col, max
from pyspark.sql.functions import format_string
from pyspark.sql.functions import substring
from pyspark.sql.functions import concat_ws
from pyspark.sql.functions import concat
from pyspark.sql.functions import to_timestamp
from pyspark.sql.functions import lit
from pyspark.sql.functions import to_utc_timestamp
from pyspark.sql.functions import expr
from pyspark.sql.functions import regexp_replace
from pyspark.sql.functions import instr
from pyspark.sql.functions import row_number
from pyspark.sql.window import Window
from pyspark.mllib.classification import LogisticRegressionWithLBFGS
from pyspark.mllib.evaluation import BinaryClassificationMetrics
from pyspark.mllib.util import MLUtils
from pyspark.mllib.regression import LabeledPoint
from pyspark.ml.linalg import Vectors
from pyspark.mllib.evaluation import MulticlassMetrics
import random as rnd
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import OneHotEncoder
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StandardScaler
from pyspark.ml.classification import LogisticRegression


from datetime import datetime
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [0]:
data_BASE_DIR = "dbfs:/mnt/mids-w261-joined/"
display(dbutils.fs.ls(f"{data_BASE_DIR}"))

path,name,size,modificationTime
dbfs:/mnt/mids-w261-joined/YEAR=2015/,YEAR=2015/,0,1658437611000
dbfs:/mnt/mids-w261-joined/YEAR=2016/,YEAR=2016/,0,1658437611000
dbfs:/mnt/mids-w261-joined/YEAR=2017/,YEAR=2017/,0,1658437611000
dbfs:/mnt/mids-w261-joined/YEAR=2018/,YEAR=2018/,0,1658437611000
dbfs:/mnt/mids-w261-joined/YEAR=2019/,YEAR=2019/,0,1658437611000
dbfs:/mnt/mids-w261-joined/YEAR=2020/,YEAR=2020/,0,1658437611000
dbfs:/mnt/mids-w261-joined/YEAR=2021/,YEAR=2021/,0,1658437611000
dbfs:/mnt/mids-w261-joined/_SUCCESS,_SUCCESS,0,1658437611000
dbfs:/mnt/mids-w261-joined/_committed_1995221853517722120,_committed_1995221853517722120,38,1658437611000


In [0]:
# Read in training and test data

df_2015 = spark.read.parquet(f"{data_BASE_DIR}YEAR=2015/*")
df_2016 = spark.read.parquet(f"{data_BASE_DIR}YEAR=2016/*")
df_2017 = spark.read.parquet(f"{data_BASE_DIR}YEAR=2017/*")
df_2018 = spark.read.parquet(f"{data_BASE_DIR}YEAR=2018/*")
df_2019 = spark.read.parquet(f"{data_BASE_DIR}YEAR=2019/*")
df_2020 = spark.read.parquet(f"{data_BASE_DIR}YEAR=2020/*")
df_2021 = spark.read.parquet(f"{data_BASE_DIR}YEAR=2021/*")

In [0]:
display(df_2015)

_utc_dept_ts,_utc_dept_minus2_ts,_utc_dept_actual_ts,_utc_arr_ts,_utc_arr_actual_ts,_dep_time_str,_local_dept_ts,_local_dept_minus2_ts,_local_dept_actual_ts,_local_at_src_airport_arr_ts,_local_at_src_airport_arr_actual_ts,QUARTER,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,FL_DATE,OP_UNIQUE_CARRIER,OP_CARRIER_AIRLINE_ID,OP_CARRIER,TAIL_NUM,OP_CARRIER_FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN_AIRPORT_SEQ_ID,ORIGIN_CITY_MARKET_ID,ORIGIN,ORIGIN_CITY_NAME,ORIGIN_STATE_ABR,ORIGIN_STATE_FIPS,ORIGIN_STATE_NM,ORIGIN_WAC,DEST_AIRPORT_ID,DEST_AIRPORT_SEQ_ID,DEST_CITY_MARKET_ID,DEST,DEST_CITY_NAME,DEST_STATE_ABR,DEST_STATE_FIPS,DEST_STATE_NM,DEST_WAC,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,DEP_DELAY_NEW,DEP_DEL15,DEP_DELAY_GROUP,DEP_TIME_BLK,TAXI_OUT,WHEELS_OFF,WHEELS_ON,TAXI_IN,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,ARR_DELAY_NEW,ARR_DEL15,ARR_DELAY_GROUP,ARR_TIME_BLK,CANCELLED,CANCELLATION_CODE,DIVERTED,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,FLIGHTS,DISTANCE,DISTANCE_GROUP,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,FIRST_DEP_TIME,TOTAL_ADD_GTIME,LONGEST_ADD_GTIME,origin_airport_iata,origin_airport_tz,origin_airport_type,origin_airport_elevation,origin_airport_iso_country,origin_airport_iso_region,origin_airport_ws_station_id,dest_airport_iata,dest_airport_tz,dest_airport_type,dest_airport_elevation,dest_airport_iso_country,dest_airport_iso_region,dest_airport_ws_station_id,origin_weather_Station,origin_weather_Datehour,origin_weather_Avg_Elevation,origin_weather_Avg_HourlyAltimeterSetting,origin_weather_Avg_HourlyDewPointTemperature,origin_weather_Avg_HourlyDryBulbTemperature,origin_weather_Avg_HourlyPressureChange,origin_weather_Avg_HourlyRelativeHumidity,origin_weather_Avg_HourlySeaLevelPressure,origin_weather_Avg_HourlyStationPressure,origin_weather_Avg_HourlyVisibility,origin_weather_Avg_HourlyWetBulbTemperature,origin_weather_Avg_HourlyWindDirection,origin_weather_Avg_HourlyWindGustSpeed,origin_weather_Avg_HourlyWindSpeed,origin_weather_Avg_Precip_Double,origin_weather_Trace_Rain,origin_weather_NonZero_Rain,origin_weather_HourlyPressureTendency_Increasing,origin_weather_HourlyPressureTendency_Decreasing,origin_weather_HourlyPressureTendency_Constant,origin_weather_Calm_Winds,origin_weather_Sky_Conditions_CLR,origin_weather_Sky_Conditions_FEW,origin_weather_Sky_Conditions_SCT,origin_weather_Sky_Conditions_BKN,origin_weather_Sky_Conditions_OVC,origin_weather_Sky_Conditions_VV,origin_weather_Present_Weather_Drizzle,origin_weather_Present_Weather_Rain,origin_weather_Present_Weather_Snow,origin_weather_Present_Weather_SnowGrains,origin_weather_Present_Weather_IceCrystals,origin_weather_Present_Weather_Hail,origin_weather_Present_Weather_Mist,origin_weather_Present_Weather_Fog,origin_weather_Present_Weather_Smoke,origin_weather_Present_Weather_Dust,origin_weather_Present_Weather_Haze,origin_weather_Present_Weather_Storm,dest_weather_Station,dest_weather_Datehour,dest_weather_Avg_Elevation,dest_weather_Avg_HourlyAltimeterSetting,dest_weather_Avg_HourlyDewPointTemperature,dest_weather_Avg_HourlyDryBulbTemperature,dest_weather_Avg_HourlyPressureChange,dest_weather_Avg_HourlyRelativeHumidity,dest_weather_Avg_HourlySeaLevelPressure,dest_weather_Avg_HourlyStationPressure,dest_weather_Avg_HourlyVisibility,dest_weather_Avg_HourlyWetBulbTemperature,dest_weather_Avg_HourlyWindDirection,dest_weather_Avg_HourlyWindGustSpeed,dest_weather_Avg_HourlyWindSpeed,dest_weather_Avg_Precip_Double,dest_weather_Trace_Rain,dest_weather_NonZero_Rain,dest_weather_HourlyPressureTendency_Increasing,dest_weather_HourlyPressureTendency_Decreasing,dest_weather_HourlyPressureTendency_Constant,dest_weather_Calm_Winds,dest_weather_Sky_Conditions_CLR,dest_weather_Sky_Conditions_FEW,dest_weather_Sky_Conditions_SCT,dest_weather_Sky_Conditions_BKN,dest_weather_Sky_Conditions_OVC,dest_weather_Sky_Conditions_VV,dest_weather_Present_Weather_Drizzle,dest_weather_Present_Weather_Rain,dest_weather_Present_Weather_Snow,dest_weather_Present_Weather_SnowGrains,dest_weather_Present_Wea

In [0]:
df_2015.count()

Out[6]: 5799749

In [0]:
df_2016.count()

Out[8]: 5596190

In [0]:
df_2017.count()

Out[9]: 5651381

In [0]:
df_2018.count()

Out[10]: 7189556

In [0]:
df_2019.count()

Out[11]: 7393347

In [0]:
df_2020.count()

Out[12]: 4671672

In [0]:
df_2021.count()

Out[13]: 5969738

In [0]:
total = df_2015.count() + df_2016.count() + df_2017.count() + df_2018.count() + df_2019.count() + df_2020.count() + df_2021.count()

In [0]:
total

Out[15]: 42271633